# Are users home locations in Vulnerable Locations?

In [13]:
import pandas as pd; import numpy as np; from multiprocessing import Pool, Manager; import numpy as np; import geopandas as gpd
import matplotlib.pyplot as plt; import seaborn as sns
import matplotlib, os, json, sys, time, datetime
from bson import json_util

In [14]:
input_directory  = "/data/chime/geo2/PROCESSED/FL/martin_palm_beach_counties_Stage2/"
output_directory = "/data/chime/geo2/PROCESSED/FL/martin_palm_beach_counties_Stage3/"
zoneGeometry     = "../EvacuationZones/Florida/martin_palm_et_al.shp"

#### Import the ZoneA Geometry

In [15]:
import fiona, shapely; from osgeo import ogr; from shapely.geometry import mapping, shape
c = fiona.open(zoneGeometry,'r')
pol = c.next(); zone = shape(pol['geometry']).buffer(0); zone.is_valid

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Collection.__next__() is buggy and will be removed in Fiona 2.0. Switch to `next(iter(collection))`.
  This is separate from the ipykernel package so we can avoid doing imports until


True

#### Import the user metadata DF (Phasing this out)

In [16]:
_user_meta = pd.read_json(input_directory+'/temporal_clustered_user_meta.json')

In [17]:
_user_meta = _user_meta.sort_index()
_user_meta.head(3)

,home_cluster,home_cluster_coords,tweets,uid,user
0,11.0,"{""coordinates"": [-80.20819619047614, 25.861914...",10784,249831776,TotalTrafficMIA
1,134.0,"{""coordinates"": [-78.01829999999997, 33.914999...",3523,4754740136,ebbtidebot
2,1.0,"{""coordinates"": [-80.0692510000025, 26.3507769...",3213,174303316,ClickWhisperer


## 0. Import all of the individual user dataframes

In [18]:
users_in = sorted(os.listdir(input_directory))
users_in = [x for x in users_in if x != "temporal_clustered_user_meta.json"]
print("Found {0} users in {1}".format(len(users_in), input_directory))

Found 363 users in /data/chime/geo2/PROCESSED/FL/martin_palm_beach_counties_Stage2/


In [19]:
def loader_function(args):
    uFile, path, q = args
    u = json.load(open(path+"/"+uFile,'r'))
    tweets = []
    for t in u['features']:
        if t['geometry']:
            t['properties']['geometry'] = shape(t['geometry'])
        t['properties']['date'] = pd.Timestamp(t['properties']['date'])
        tweets.append(t['properties'])
    q.put(1)
    return gpd.GeoDataFrame(tweets)

In [20]:
#Parallel runtime
p = Pool(30)
m = Manager()
q = m.Queue()

args = [(i, input_directory, q) for i in users_in]
result = p.map_async(loader_function, args)

# monitor loop
while True:
    if result.ready():
        break
    else:
        size = q.qsize()
        sys.stderr.write("\rProcessed: {0}, {1:.3g}%".format(size, size/len(args)*100))
        time.sleep(0.5)
sys.stderr.write("\rProcessed: {0}, {1:.3g}%".format(q.qsize(), q.qsize()/len(args)*100))
users = result.get()
p.close()

Processed: 363, 100%%

In [21]:
users.sort( key=lambda x: len(x), reverse=True ) 

`users` is an array of user dataframes. Now find which users have _home locations_ in Zone A

In [22]:
x = users[10]
print(len(x))
x.head(1)

756


,cluster,cluster_center,coords,date,day_cluster,geo_delta,geometry,home_cluster_id,speed,text,time_delta,tweet_id,uid,user
0,1,"{""coordinates"": [-76.31451879999999, 36.867423...","[-76.3145188, 36.8674231]",2016-09-23 09:44:35+00:00,2,NaN,POINT (-76.3145188 36.8674231),8.0,NaN,Can you recommend anyone for this #job in #Nor...,NaN,779255131376197632,2587789764,WorkWithSHC


In [23]:
def get_home_cluster_center(userDF):
    return shape(json.loads(userDF.query('cluster=='+str(userDF.home_cluster_id.values[0])).cluster_center.values[0]))

get_home_cluster_center(users[0]).within(zone)

False

## 1. Determine who's home cluster center is in ZONE

In [24]:
def insideZone(p):
    if p==None:
        return False
    else:
        return p.within(zone)

In [25]:
vuln = []
non_vuln = []
for idx, u in enumerate(users):
    if (insideZone(get_home_cluster_center(u))):
        vuln.append(u)
    else:
        non_vuln.append(u)
    sys.stderr.write("\r"+str(idx+1))
sys.stderr.write("\rDone...")
sys.stderr.write("Identified {0} vulnerable users and {1} non-vulnerable".format(len(vuln),len(non_vuln)))

Done...Identified 13 vulnerable users and 350 non-vulnerable

In [26]:
if not os.path.exists(output_directory):
    os.mkdir(output_directory)

## 2. Write out just the GeoVulnerable, just in case we need them for something later

In [27]:
def safe_mapping(p):
    if p==None or np.isnan(p).any():
        return None
    else:
        return mapping(p)
def safe_json_export(args):
    df, path = args
    df = df.copy()
    uName = df.head(1).user.values[0].lower()
    df['date'] = df['date'].apply(lambda t: datetime.datetime.strftime(t,'%Y-%m-%dT%H:%M:%SZ'))

    clean = df.where((pd.notnull(df)), None)
    geojson = {"type":"FeatureCollection","features":[]}
    for _, row in clean.iterrows():
        geom = safe_mapping(row.geometry)
        feature = {'type':'Feature',
                   'geometry':geom,
                   'properties':row.to_dict()
                    }
        del feature['properties']['geometry']
        geojson['features'].append(feature)
    
    with open(path+"/"+uName+'.geojson','w') as oFile:
        json.dump(geojson, oFile) 

In [28]:
for idx, u in enumerate(vuln):
    safe_json_export((u,output_directory))
    sys.stderr.write("\r{0} processed".format(idx+1))

13 processed

# 3. Prepare for Analysis

All of these users should already exist in a format prepared for visualizing, these users can be pasted into Google Sheets

In [29]:
for uName in sorted([u.user[0].lower() for u in vuln]):
    print(uName)

3idinfantryvet
clickwhisperer
drsherryroth
engstrom_roger
fuzzydogphotos
itsme_raquel_
karlykane
msunitedam
nica225
pipemanradio
shrimptank
swimminglisa
valeriesteblyuk


In [30]:
len(vuln)

13

In [32]:
def good_storm_tweets(userDF):
    return len(userDF.query("date > 201610040000 & date < 201610080000")) 
    
good_data = []
res = []
for idx, u in enumerate(vuln):
    if good_storm_tweets(u) > 2:
        good_data.append(u)
    
    sys.stderr.write("\r"+str(idx))

sys.stderr.write("\r"+str(len(good_data)))
for uName in sorted([u.user[0].lower() for u in good_data]):
    print(uName)

3idinfantryvet
clickwhisperer
drsherryroth
engstrom_roger
fuzzydogphotos
itsme_raquel_
nica225
pipemanradio
shrimptank
valeriesteblyuk


10

# Create rules file from UID